In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime
from statistics import mean
import math as m

In [ ]:
def smoothing(data):
    series = data
    # Tail-rolling average transform
    rolling = series.rolling(window=8)
    rolling_mean = rolling.mean()
    return rolling_mean

In [ ]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [ ]:
def mageCalculation(df, std=1):
    
    #extracting glucose values and incdices
    glucs = df['GlucoseValue'].to_list()
    indices = [1*i for i in range(len(glucs))]
    stdev = std
    
    # detection of local minima and maxima
    x = indices
    gvs = glucs
    # local min & max
    a = np.diff(np.sign(np.diff(gvs))).nonzero()[0] + 1      
    # local min
    valleys = (np.diff(np.sign(np.diff(gvs))) > 0).nonzero()[0] + 1 
    # local max
    peaks = (np.diff(np.sign(np.diff(gvs))) < 0).nonzero()[0] + 1         
    # +1 due to the fact that diff reduces the original index number

    #storing the local minima and maxima to identify and remove turning points
    excursion_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(len(peaks)):
        excursion_points.loc[k] = [peaks[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["P"]
        k+=1

    for i in range(len(valleys)):
        excursion_points.loc[k] = [valleys[i]] + [df['Display Time'][k]] + [df['GlucoseValue'][k]] + ["V"]
        k+=1

    excursion_points = excursion_points.sort_values(by=['Index'])
    excursion_points = excursion_points.reset_index(drop=True)
    # display(excursion_points)


    # selecting turning points
    turning_points = pd.DataFrame(columns=['Index', 'Timestamp', 'GlucoseValue', 'Type'])
    k=0
    for i in range(stdev,len(excursion_points.Index)-stdev):
        positions = [i-stdev,i,i+stdev]
        for j in range(0,len(positions)-1):
            if(excursion_points.Type[positions[j]] == excursion_points.Type[positions[j+1]]):
                if(excursion_points.Type[positions[j]]=='P'):
                    if excursion_points.GlucoseValue[positions[j]]>=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1
                else:
                    if excursion_points.GlucoseValue[positions[j]]<=excursion_points.GlucoseValue[positions[j+1]]:
                        turning_points.loc[k] = excursion_points.loc[positions[j]]
                        k+=1
                    else:
                        turning_points.loc[k] = excursion_points.loc[positions[j+1]]
                        k+=1

    if len(turning_points.index)<10:
        turning_points = excursion_points.copy()
        excursion_count = len(excursion_points.index)
    else:
        excursion_count = len(excursion_points.index)/2



    turning_points = turning_points.drop_duplicates(subset= "Index", keep= "first")
    turning_points=turning_points.reset_index(drop=True)
    excursion_points = excursion_points[excursion_points.Index.isin(turning_points.Index) == False]
    excursion_points = excursion_points.reset_index(drop=True)
        # display(turning_points)

    # calculating the MAGE score
    mage = turning_points.GlucoseValue.sum()/excursion_count
    

    return round(mage,3), excursion_count

In [ ]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/consolidatedDataForPackage.csv")
data['Display Time'] = data['Display Time'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

for subjectId, df in tqdm(data.groupby('subjectId')):
    
    df = fullDay(df)
    length = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    length = length.round("d")
    days = length.days

    df['GlucoseValue'] = smoothing(df['GlucoseValue'])
    df = df[df['GlucoseValue'].notna()]
    df = df.reset_index(drop=True)

    dates = []
    for i in range(len(df.index)):
        dates.append(df['Display Time'][i].date())
    df['Date'] = dates

    mage_daily = []
    excursions = 0
    for Date, xx in df.groupby('Date'):
        xx = xx.reset_index(drop=True)
        mage, e = mageCalculation(xx,1)
        mage_daily.append(mage)
        excursions = excursions + e
    print(subjectId, round(mean(mage_daily),3), m.floor(excursions), sep="\t")

